In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import tensorflow as tf
import os
from PIL import Image, ImageDraw

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
RAW_DIRECTORY = './drive/My Drive/dataset/raw'
INPUT_DIRECTORY = './drive/My Drive/dataset/input'
OUTPUT_DIRECTORY = './drive/My Drive/dataset/output'
TRAIN_DIRECTORY = './drive/My Drive/dataset/train'

In [4]:
def preprocess(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image)
  img_dim = min(image.shape[0], image.shape[1])
  dim = (img_dim, img_dim)
  image = image[0:img_dim, 0:img_dim]
  resized = cv.resize(np.asarray(image), (256, 256), interpolation = cv.INTER_AREA)
  return resized

In [5]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

In [6]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [7]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [8]:
# normalizing the images to [-1, 1]
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1
  return input_image, real_image

In [9]:
@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

In [10]:
counter = 0
for filename in os.listdir(RAW_DIRECTORY):
  RAW_IMAGE_PATH = RAW_DIRECTORY + '/' + filename
  img = preprocess(RAW_IMAGE_PATH)
  counter = counter + 1
  tf.keras.preprocessing.image.save_img(INPUT_DIRECTORY + '/' + str(counter) + '.jpg', img)

In [16]:
def extract_edges(path):
  img = cv.imread(path, cv.IMREAD_COLOR)
  edges = cv.Canny(img, 200, 400)
  edges = ~edges
  return edges

In [17]:
for filename in os.listdir(INPUT_DIRECTORY):
  INPUT_IMAGE_PATH = INPUT_DIRECTORY + '/' + filename
  OUTPUT_IMAGE_PATH = OUTPUT_DIRECTORY + '/' + filename
  img = extract_edges(INPUT_IMAGE_PATH)
  cv.imwrite(OUTPUT_IMAGE_PATH, img)

In [18]:
def stack(path1, path2):
  im = cv.imread(path1)
  img = cv.imread(path2, 0)  
  imgc = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
  both = np.concatenate((im,imgc), axis=1)   #1 : horz, 0 : Vert. 
  return both

In [19]:
for filename in os.listdir(INPUT_DIRECTORY):
  both = stack(INPUT_DIRECTORY + '/' + filename, OUTPUT_DIRECTORY + '/' + filename)
  cv.imwrite(TRAIN_DIRECTORY + '/' + filename, both)